In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle



    Only loading the 'en' tokenizer.



In [2]:
PATH='data/predictHappiness/'
!ls {PATH}

all						sample_submission.csv  trn
models						test.csv	       val
PredictHappinessModel_17_11_28_03_ensemble.csv	tmp
PredictHappinessModel_Final_ensemble.csv	train.csv


In [3]:
trn = pd.read_csv(PATH+"train.csv")
tst = pd.read_csv(PATH+"test.csv")

Builds the structure of the folders I used

In [4]:
#for i in trn.Is_Response.unique():
#    trn_dir_path = f'{PATH}trn/{i.replace(" ","_")}'
#    val_dir_path = f'{PATH}val/{i.replace(" ","_")}'
#    !rm -r {trn_dir_path}
#    !rm -r {val_dir_path}
#    os.makedirs(trn_dir_path, exist_ok=True)
#    os.makedirs(val_dir_path, exist_ok=True)
#!rm -r {PATH}all/trn
#!rm -r {PATH}all/val
#os.makedirs(f'{PATH}all/trn', exist_ok=True)
#os.makedirs(f'{PATH}all/val', exist_ok=True)
#os.makedirs(f'{PATH}all/tst', exist_ok=True)
#os.makedirs(f'{PATH}models', exist_ok=True)
#os.makedirs(f'{PATH}/models/Stage1Model', exist_ok=True)
#os.makedirs(f'{PATH}/models/Stage2Model', exist_ok=True)

This saves the csv in individual files in folders that split them into happy and not_happy


In [5]:
#for i in range(trn.values[:,1].shape[0]):
#    dset = 'trn' if random.random()>0.05 else 'val'
#    f = open(f"{PATH}all/{dset}/"+trn.values[i,0]+".txt", 'w')
#    f.write(trn.values[i,1])
#    f.close()
#    f = open(f"{PATH}{dset}/"+trn.Is_Response[i].replace(" ","_")+"/"+trn.values[i,0]+".txt", 'w')
#    f.write(trn.values[i,1])
#    f.close()

In [6]:
#for i in range(tst.values[:,1].shape[0]):
#    f = open(PATH+"all/tst/"+tst.values[i,0]+".txt", 'w')
#    f.write(tst.values[i,1])
#    f.close()

Pull all the words in using the spacy tokenizer and set lower to True (One step I wish I had more time to play with)

In [4]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [5]:
bs=64; bptt=70

In [6]:
FILES = dict(train="all/trn/", validation="all/val/", test="all/tst/")

min_freq is set to 10 so anything that occurs less than 10 times won't show up.  I would like to increase this to 25 or 50 but 10 seemed to do the job. 

In [7]:
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [8]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [9]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

The goal of this first model is just to be able to learn what words make sense together and to be able to build a coherent sentence.  This ensures that the embedding weights are reasonable.  This was a place that I found made a big difference in my final score.  When I trained this a few times I was getting ok scores, but when I trained it more and added the cycle_mult which brings in cyclicle annealing I was able to get some of my best scores.  One way I know I could get a higher score is to add another model to my final ensemble with one more round of training.  This did very well by itself, but I never was able to add it to the final ensemble because the weights were lost and I didn't have time to retrain. 

In [10]:
#multiplier=1 #play around with this
#dropouti=0.05*multiplier
#dropout=0.05*multiplier
#wdrop=0.1*multiplier
#dropoute=0.02*multiplier
#dropouth=0.05*multiplier
#learner = md.get_model(opt_fn, em_sz, nh, nl,
#               dropouti=dropouti, dropout=dropout, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
#learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
#learner.clip=0.3

In [44]:
#learner.fit(3e-3, 7, wds=1e-6, cycle_len=1, cycle_mult=2, cycle_save_name="PtH_Stage1_cyclemult")

HBox(children=(IntProgress(value=0, description='Epoch', max=127), HTML(value='')))

[ 0.       4.65618  4.6425 ]                                  
[ 1.       3.83107  3.76976]                                  
[ 2.       3.68609  3.64447]                                  
[ 3.       3.59806  3.52884]                                  
[ 4.       3.47371  3.44122]                                  
[ 5.       3.40985  3.38482]                                  
[ 6.       3.36285  3.37347]                                  
[ 7.       3.45098  3.41205]                                  
[ 8.       3.40512  3.37419]                                  
[ 9.       3.36023  3.34766]                                  
[ 10.        3.33518   3.31689]                               
[ 11.        3.2799    3.28944]                               
[ 12.        3.24986   3.28134]                               
[ 13.        3.22152   3.27062]                               
[ 14.        3.21745   3.26814]                               
[ 15.        3.35704   3.33033]                        

In [55]:
#learner.load_cycle("PtH_Stage1_cyclemult", 6)

Saving this as weights for md3 which is model 1 for the ensemble

In [56]:
#learner.save_encoder('PtH_Stage1_cyclemult_newtest')

Then I am going to train it another long while and save the weights again for another.  I believe if I kept doing this I would keep seeing marginal improvements but decided to just do two for now. 

In [63]:
#learner.fit(3e-3, 7, wds=1e-6, cycle_len=1, cycle_mult=2, cycle_save_name="PtH_Stage1_cyclemul2")

HBox(children=(IntProgress(value=0, description='Epoch', max=127), HTML(value='')))

[ 0.       3.0863   3.18537]                                  
[ 1.       3.15896  3.203  ]                                  
[ 2.       3.0592   3.18375]                                  
[ 3.       3.20134  3.22471]                                  
[ 4.       3.12271  3.19833]                                  
[ 5.       3.05572  3.1796 ]                                  
[ 6.       3.01982  3.17836]                                  
[ 7.       3.19351  3.22282]                                  
[ 8.       3.16917  3.22355]                                  
[ 9.       3.16295  3.2084 ]                                  
[ 10.        3.11239   3.19955]                               
[ 11.        3.06802   3.18485]                               
[ 12.        3.02773   3.18085]                               
[ 13.        3.01139   3.17454]                               
[ 14.        3.00529   3.17281]                               
[ 15.        3.19182   3.22998]                        

In [11]:
#learner.load_cycle("PtH_Stage1_cyclemul2", 6)

In [66]:
#learner.save_encoder('PtH_Stage1_cyclemult_newtest_secondRun')

In [12]:
#learner.load_encoder("PtH_Stage1_cyclemult_newtest_secondRun")

So now I have two model weights saved as PtH_Stage1_cyclemult_newtest_secondRun and PtH_Stage1_cyclemult_newtest.  This will be the starting point for my final weights which are specifically trained to learn happy and unhappy reviews.  The current weights are just meant to build reviews themselves which is demonstrated below.  

In [19]:
#TEXT = pickle.load(open(f'{PATH}models/TEXT3.pkl','rb'))

In [12]:
m=learner.model
ss="""This hotel is so """
s = [spacy_tok(ss)]
t = TEXT.numericalize(s)
' '.join(s[0])

'This hotel is so'

In [13]:
m[0].bs = 1
m.eval()
m.reset()
res,*_ = m(t)

m[0].bs=bs

In [14]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['close',
 'nice',
 'much',
 'small',
 'convenient',
 '-',
 'good',
 '<unk>',
 'well',
 'expensive']

In [15]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

This hotel is so  

close to the airport , you can get to the airport , and the hotel is very close to the airport . <eos> i stayed at the hotel for - nights in early september . the hotel is in a great location , close to the subway , and the ...


This group of words makes sense as a review and I could reasonably see this being something on a website. Definitely not perfect, but also reasonable enough that it could be just human error. 

Had to create a torchtext dataset since fastai is built to work very closely with torchtext.  This ended up being much easier than I had guessed.  

In [10]:
class PredictHappinessDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [("Description", text_field), ("Is_Response", label_field)]
        examples = []
        #for label in ['happy', 'not_happy']:
        #    for fname in iglob(os.path.join(path, label, '*.txt')):
        #        with open(fname, 'r') as f: text = f.readline()
        for i in range(trn.values[:,1].shape[0]):
            text = trn.Description[i]
            label = trn.Is_Response[i]
            examples.append(data.Example.fromlist([text, label], fields))#[fields[1], fields[-1]]))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.Description)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [11]:
PH_LABEL = data.Field(sequential=False)

In [12]:
splits = PredictHappinessDataset.splits(TEXT, PH_LABEL, PATH, train='trn', test='val')

Here is where I take the model from above and teach it how to determine happy vs unhappy

In [13]:
#Description and label name are coming from the dataset I created above these values are being fed in from the CSV
#This is an importand difference from reading directly from the CSV. 
md2 = TextData.from_splits(PATH, splits, bs=bs, text_name="Description", label_name="Is_Response")

In [14]:
multiplier=1
dropout=0.1*multiplier
dropouti=0.4*multiplier
wdrop=0.5*multiplier
dropoute=0.05*multiplier
dropouth=0.3*multiplier

m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl,dropout=dropout, dropouti=dropouti, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('PtH_Stage1_cyclemult_newtest_secondRun')

m3.clip=25.
lrs=np.array([1e-4,1e-3,1e-2])

In [27]:
#this is from the second run through doubling dictionary traintime
#m3.freeze_to(-1)
#m3.fit(lrs, 4, metrics=[accuracy])
#m3.unfreeze()
#m3.fit(lrs, 4, metrics=[accuracy], cycle_len=1, cycle_mult=2, cycle_save_name="m3cycles")

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

  5%|▌         | 31/608 [00:03<01:08,  8.36it/s, loss=0.46] 


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/THC/generic/THCStorage.cu:66

In [101]:
#m3.save_encoder("m3_weights")

In [22]:
#m3.load_encoder("m3_weights")

In [15]:
m3.load("m3_model2")

In [16]:
multiplier=1
dropout=0.1*multiplier
dropouti=0.4*multiplier
wdrop=0.5*multiplier
dropoute=0.05*multiplier
dropouth=0.3*multiplier

m4 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl,dropout=dropout, dropouti=dropouti, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
m4.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m4.load_encoder('PtH_Stage1_cyclemult_newtest')

m4.clip=25.

In [97]:
#m4.freeze_to(-1)
#m4.fit(lrs, 4, metrics=[accuracy])
#m4.unfreeze()
#m4.fit(lrs, 4, metrics=[accuracy], cycle_len=1, cycle_mult=2, cycle_save_name="m4cycle")

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[ 0.       0.40545  0.32259  0.87009]                        
[ 1.       0.40197  0.36688  0.85231]                        
[ 2.       0.43458  0.27963  0.8843 ]                        
[ 3.       0.4265   0.28263  0.88567]                        



HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       0.31343  0.24236  0.90365]                        
[ 1.       0.28469  0.22289  0.91142]                        
[ 2.       0.26625  0.2234   0.91293]                        
[ 3.       0.26935  0.22339  0.91224]                        
[ 4.       0.25794  0.2094   0.91856]                        
[ 5.       0.23912  0.20621  0.92018]                        
[ 6.       0.23435  0.20324  0.92126]                        
[ 7.       0.25121  0.19627  0.92301]                        
[ 8.       0.24058  0.19371  0.92254]                        
[ 9.       0.23593  0.1848   0.92768]                        
[ 10.        0.22597   0.17993   0.92948]                    
[ 11.        0.22256   0.17637   0.93118]                    
[ 12.        0.2095    0.17469   0.93177]                    
[ 13.        0.21815   0.17338   0.9321 ]                    
[ 14.        0.20402   0.17356   0.93187]                    



In [102]:
#m4.save_encoder("m4_weights")

In [17]:
m4.load("m4_model")

In [18]:
multiplier=1
dropout=0.1*multiplier
dropouti=0.4*multiplier
wdrop=0.5*multiplier
dropoute=0.05*multiplier
dropouth=0.3*multiplier

m5 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl,dropout=dropout, dropouti=dropouti, wdrop=wdrop, dropoute=dropoute, dropouth=dropouth)
m5.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m5.load_encoder('PtH_Stage1_cyclemult_newtest_thirdRun')

m5.clip=25.

In [19]:
m5.load("m5_model")

In [20]:
submission = pd.DataFrame(columns=['User_ID','Is_Response'])

In [21]:
m3m = m3.model 
m4m = m4.model
m5m = m5.model
m3m[0].bs = 1
m4m[0].bs = 1
m5m[0].bs = 1
for i in range(tst.values[:,1].shape[0]):
    ss = tst["Description"][i]
    s = [spacy_tok(ss)]
    t = TEXT.numericalize(s)
   
    m3m.eval()
    m4m.eval()
    m5m.eval()
    m3m.reset()
    m4m.reset()
    m5m.reset()
    resm3m,*_ = m3m(t)
    resm4m,*_ = m4m(t)
    resm5m,*_ = m5m(t)
    
    res = resm3m+resm4m+resm5m
    user_id = tst["User_ID"][i]
    is_response = PH_LABEL.vocab.itos[to_np(torch.topk(res[-1], 1)[1])[0]]
    submission = submission.append({"User_ID":user_id, 'Is_Response':is_response.replace(" ","_")},ignore_index=True)

In [22]:
submission.to_csv(PATH+"PredictHappinessModel_Final_ensemble_17_11_29.csv", index=False)